![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Taller: Análisis de sentimientos y técnicas de NLP

En este taller podrán poner en práctica sus conocimientos sobre las diferentes técnicas para el procesamiento de lenguaje natural. El taller está constituido por 5 puntos, en los cuales deberan seguir las intrucciones de cada numeral para su desarrollo.

## Datos predicción sentimientos de viajeros en Twitter

En este taller se usará el conjunto de datos de sentimientos sobre distintas aerolíneas de EE.UU. provenientes de Twitter. Cada observación contiene si el sentimiento de los tweets es positivo, neutral o negativo teniendo en cuenta distintas variables como aerolínea y las razones de los sentimientos negativos (como "retraso en el vuelo" o "servicio grosero"). El objetivo es predecir el sentimiento asociado a cada tweet. Para más detalles pueden visitar el siguiente enlace: [datos](https://www.kaggle.com/crowdflower/twitter-airline-sentiment).

### Librerías a Importar

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
# Importación de librerías
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from imblearn.over_sampling import SMOTE

In [ ]:
# Lectura de la información de archivo .zip
tweets = pd.read_csv('https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/Tweets.zip', index_col=0)

# Visualización dataset
tweets.head()

In [ ]:
# Impresión tamaño del cojunto de datos
tweets.shape

### Análisis descriptivo

In [5]:
# Cuenta de tweets por cada sentimiento
tweets['airline_sentiment'].value_counts()

airline_sentiment
negative    9178
neutral     3099
positive    2363
Name: count, dtype: int64

In [6]:
# Cuenta de tweets por cada aerolínea
tweets['airline'].value_counts()

airline
United            3822
US Airways        2913
American          2759
Southwest         2420
Delta             2222
Virgin America     504
Name: count, dtype: int64

In [ ]:
# Plot con cuenta de tweets por cada aerolínea y sentimiento
pd.crosstab(index = tweets["airline"],columns = tweets["airline_sentiment"]).plot(kind='bar',figsize=(10, 6),alpha=0.5,rot=0,stacked=True,title="Sentiminetos por aerolínea")

### Separación de Datos en Conjunto de Entrenamiento y Evaluación

In [4]:
# Separación de variables predictoras (X) y de variable de interés (y)
X = tweets['text']
y = tweets['airline_sentiment'].map({'negative':0,'neutral':1,'positive':2})

In [5]:
# Separación de datos en set de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Punto 1 - Uso de CountVectorizer

En la celda 1 creen un modelo de random forest con la libreria sklearn que prediga el sentimiento de los tweets usando los set de entrenamiento y test definidos anteriormente. Usen la función **CountVectorizer** y presenten el desempeño del modelo con la métrica del acurracy.

Recuerden que el preprocesamiento que se haga sobre los datos de entrenamiento  (*.fit_transform()*) deben ser aplicado al set de test (*.transform()*).

In [6]:
# Aplicación de CountVectorizer para convertir el texto en una matriz de conteo
count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

In [7]:
# Balanceo de clases con SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_count, y_train)

In [8]:

# Entrenamiento y predicción del modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_res, y_train_res)
y_pred_rf = rf_model.predict(X_test_count)

In [ ]:
# Evaluación del modelo con el conjunto de test
accuracy_RF = accuracy_score(y_test, y_pred_rf)
Reporte_RF = classification_report(y_test, y_pred_rf)
Matriz_conf_RF = confusion_matrix(y_test, y_pred_rf)

# Mostramos los resultados
print(f" Accuracy del modelo: {accuracy_RF:.4f}")
print("\n Reporte de clasificación:\n", Reporte_RF)
print("\n Matriz de confusión:\n", Matriz_conf_RF)

 Accuracy del modelo: 0.7649

 Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.83      0.89      0.86      3085
           1       0.58      0.54      0.56       984
           2       0.69      0.54      0.60       763

    accuracy                           0.76      4832
   macro avg       0.70      0.66      0.68      4832
weighted avg       0.76      0.76      0.76      4832


 Matriz de confusión:
 [[2751  252   82]
 [ 347  536  101]
 [ 224  130  409]]


### Descripción del Procedimiento y Conclusiones

<p style="text-align: justify;">
En este punto se transforman los los textos de los tweets en una matriz numérica según la frecuencia de las palabras mediante CountVectorizer. Luego, se aplica SMOTE al conjunto de entrenamiento para balancear las clases minoritarias (<code>'neutral'</code> y <code>'positive'</code>). Con los datos balanceados, se entrena un modelo de Random Forest y, finalmente, se evalúa su desempeño en el conjunto de prueba utilizando métricas como accuracy, reporte de clasificación y matriz de confusión.<p>
<p style="text-align: justify;">
En general modelo de obtuvo una exactitud del 76.49%. La clase 0 (<code>'negative'</code>) presentó una precisión de 0.83 y recall de 0.89, mientras que las clases 1 y 2 tuvieron un recall más bajo (0.54 en ambos casos). La matriz de confusión confirma más errores en estas últimas clases. El desbalance de datos podría influir en estos resultados.<p>

### Punto 2 - Eliminación de Stopwords
<p style="text-align: justify;">
En la celda 2 creen un modelo de random forest con la libreria sklearn que prediga el sentimiento de los tweets usando los set de entrenamiento y test definidos anteriormente. Usen la función CountVectorizer, **eliminen stopwords** y presenten el desempeño del modelo con la métrica del acurracy.<p>

Recuerden que el preprocesamiento que se haga sobre los datos de entrenamiento  (*.fit_transform()*) deben ser aplicado al set de test (*.transform()*).

In [ ]:
# Limpieza de texto
lemmatizer = WordNetLemmatizer()                                          # Inicializamos el lematizador
def limpiar_texto(texto):
    stop_words = set(stopwords.words('english'))                          # Lista de stopwords 
    tokens = wordpunct_tokenize(texto)                                    # Separamos el texto por palabras (tokens)
        
    tokens_filtrados = [p for p in tokens if p.lower() not in stop_words and p.isalpha()]   
    return ' '.join(tokens_filtrados)

# Aplicación de la función de limpieza a los textos
X_train_limpio = X_train.apply(limpiar_texto)
X_test_limpio = X_test.apply(limpiar_texto)

In [ ]:
# Convertimos los textos a vectores usando CountVectorizer
vectorizador = CountVectorizer()
X_train_vec = vectorizador.fit_transform(X_train_limpio)
X_test_vec = vectorizador.transform(X_test_limpio)

In [ ]:
# Balanceo de clases con SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_vec, y_train)

In [ ]:
# Creamos y entrenamos el modelo Random Forest
modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf.fit(X_train_res, y_train_res)

# Hacemos predicciones y evaluamos el modelo
predicciones = modelo_rf.predict(X_test_vec)

# Métricas de desempeño
accuracy_StopWords = accuracy_score(y_test, predicciones)
reporte = classification_report(y_test, predicciones)
matriz_conf = confusion_matrix(y_test, predicciones)

# Mostramos los resultados
print(" Accuracy del modelo combinado:", accuracy_StopWords)
print("\n Reporte de clasificación:\n", reporte)
print("\n Matriz de confusión:\n", matriz_conf)

 Accuracy del modelo combinado: 0.7671771523178808

 Reporte de clasificación:
               precision    recall  f1-score   support

          -1       0.81      0.90      0.86      3085
           0       0.59      0.47      0.53       984
           1       0.74      0.60      0.66       763

    accuracy                           0.77      4832
   macro avg       0.71      0.66      0.68      4832
weighted avg       0.76      0.77      0.76      4832


 Matriz de confusión:
 [[2781  222   82]
 [ 437  466   81]
 [ 202  101  460]]


### Punto 3 - Lematización con verbos

En la celda 3 creen un modelo de random forest con la libreria sklearn que prediga el sentimiento de los tweets usando los set de entrenamiento y test definidos anteriormente. Usen la función CountVectorizer, **lematizen el texto con verbos** y presenten el desempeño del modelo con la métrica del acurracy.

Recuerden que el preprocesamiento que se haga sobre los datos de entrenamiento  (*.fit_transform()*) deben ser aplicado al set de test (*.transform()*).

In [26]:
# Creamos una función que limpie el texto
# Quitamos palabras vacías y lematizamos las palabras restantes
lemmatizer = WordNetLemmatizer()  # Inicializamos el lematizador
def limpiar_texto(texto):
    #stop_words = set(stopwords.words('english'))  # Lista de stopwords 
    tokens = wordpunct_tokenize(texto)  # Separamos el texto por palabras (tokens)
    
    # Filtramos las palabras: que no sean stopwords y que sean letras (no signos)
    #tokens_filtrados = [p for p in tokens if p.lower() not in stop_words and p.isalpha()]
    
    # Lematizamos usando como referencia los verbos
    lematizadas = [lemmatizer.lemmatize(p, pos='v') for p in tokens]
    
    # Unimos todo nuevamente en un texto limpio
    return ' '.join(lematizadas)

# Aplicamos esta limpieza a los datos de entrenamiento y prueba
X_train_limpio = X_train.apply(limpiar_texto)
X_test_limpio = X_test.apply(limpiar_texto)

# Convertimos los textos a vectores usando CountVectorizer
vectorizador = CountVectorizer()
X_train_vec = vectorizador.fit_transform(X_train_limpio)
X_test_vec = vectorizador.transform(X_test_limpio)


In [ ]:
# Creamos y entrenamos el modelo Random Forest
modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf.fit(X_train_vec, y_train)

# Hacemos predicciones y evaluamos el modelo
predicciones = modelo_rf.predict(X_test_vec)

# Métricas de desempeño
accuracy_lemmatizer = accuracy_score(y_test, predicciones)
reporte = classification_report(y_test, predicciones)
matriz_conf = confusion_matrix(y_test, predicciones)

# Mostramos los resultados
print(" Accuracy del modelo combinado:", accuracy_lemmatizer)
print("\n Reporte de clasificación:\n", reporte)
print("\n Matriz de confusión:\n", matriz_conf)

### Punto 4 - Multiples técnicas

En la celda 4 creen un modelo de random forest con la libreria sklearn que prediga el sentimiento de los tweets usando los set de entrenamiento y test definidos anteriormente. Usen la función **CountVectorizer, eliminen stopwords, lematizen el texto con verbos** y presenten el desempeño del modelo con la métrica del acurracy.

Recuerden que el preprocesamiento que se haga sobre los datos de entrenamiento  (*.fit_transform()*) deben ser aplicado al set de test (*.transform()*).

In [28]:
# Creamos una función que limpie el texto
# Quitamos palabras vacías y lematizamos las palabras restantes
lemmatizer = WordNetLemmatizer()  # Inicializamos el lematizador
def limpiar_texto(texto):
    stop_words = set(stopwords.words('english'))  # Lista de stopwords 
    tokens = wordpunct_tokenize(texto)  # Separamos el texto por palabras (tokens)
    
    # Filtramos las palabras: que no sean stopwords y que sean letras (no signos)
    tokens_filtrados = [p for p in tokens if p.lower() not in stop_words and p.isalpha()]
    
    # Lematizamos usando como referencia los verbos
    lematizadas = [lemmatizer.lemmatize(p, pos='v') for p in tokens_filtrados]
    
    # Unimos todo nuevamente en un texto limpio
    return ' '.join(lematizadas)

# Aplicamos esta limpieza a los datos de entrenamiento y prueba
X_train_limpio = X_train.apply(limpiar_texto)
X_test_limpio = X_test.apply(limpiar_texto)

# Convertimos los textos a vectores usando CountVectorizer
vectorizador = CountVectorizer()
X_train_vec = vectorizador.fit_transform(X_train_limpio)
X_test_vec = vectorizador.transform(X_test_limpio)

In [ ]:
# Creamos y entrenamos el modelo Random Forest
modelo_rf = RandomForestClassifier(n_estimators=100, random_state=42)
modelo_rf.fit(X_train_vec, y_train)

# Hacemos predicciones y evaluamos el modelo
predicciones = modelo_rf.predict(X_test_vec)

# Métricas de desempeño
accuracy_combined= accuracy_score(y_test, predicciones)
reporte = classification_report(y_test, predicciones)
matriz_conf = confusion_matrix(y_test, predicciones)

# Mostramos los resultados
print(" Accuracy del modelo combinado:", accuracy)
print("\n Reporte de clasificación:\n", reporte)
print("\n Matriz de confusión:\n", matriz_conf)


### Punto 5 - Comparación y análisis de resultados

En la celda 5 comparen los resultados obtenidos de los diferentes modelos (random forest) y comenten las ventajas del mejor modelo y las desventajas del modelo con el menor desempeño.

In [ ]:
modelos = ['CountVectorizer', 'StopWords', 'Lemmatizer', 'Combined']
Accuracy = [accuracy_RF, accuracy_StopWords, accuracy_lemmatizer, accuracy_combined]

fig, ax = plt.subplots(figsize=(7, 5))

# Color púrpura
purple_color = '#800080'

# Gráfico: Comparar modelos por MSE
bars = ax.bar(modelos, MSE, color=purple_color, alpha=0.7)
ax.set_title('Comparación de Modelos por MSE', fontsize=11)
ax.set_xlabel('Modelos', fontsize=11)
ax.set_ylabel('MSE', fontsize=11)
ax.grid(True, linestyle='--', alpha=0.6)

# Mostrar el valor arriba de cada barra
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2.0, height, f'{height:.4f}', 
            ha='center', va='bottom', fontsize=10)

# Ajustes finales
plt.tight_layout()
plt.show()
